# <center> <b> SMITH-WATERMAN ALGORITHM

In [11]:
import numpy as np

# Implementation

In [12]:
def smith_waterman(seq1, seq2, match_award=1, gap_penalty=-1, mismatch_penalty=-1):
    """smith_waterman
    
    Args:
    * seq1: string - first sequence as string
    * seq2: string - second sequence as string
    
    Params:
    * match_award: int = 1  - award for matching atom
    * gap_penalty: int = -1 - penalty for inserting a gap
    * mismatch_penalty: int = -1 - penalty for mismatch
    
    Returns:
    * score matrix: np.array
    * (trace1, trace2) - strings with gaps representing the best matching subsequence,
                         corresponding to two given sequences. 
    """
    
    # Returns reward if a and b are equal, and penalty otherwise 
    def match_score(a, b):
        if a == b:
            return match_award
        elif a == '-' or b == '-':
            return gap_penalty
        else:
            return mismatch_penalty

    ### Generate Score Matrix
    
    # Params
    n = len(seq1)
    m = len(seq2)  
    
    # Score Matrix
    score = np.zeros((m+1, n+1)).astype(int)
   
    # Set First Column
    for i in range(0, m + 1):
        score[i,0] = 0
    
    # Set First Row
    for j in range(0, n + 1):
        score[0,j] = 0
    
    # Set Rest of the Score Matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            diag = score[i - 1,j - 1] + match_score(seq1[j-1], seq2[i-1])
            up = score[i - 1,j] + gap_penalty
            left = score[i,j - 1] + gap_penalty
            score[i,j] = max(diag, left, up, 0)
    
    ### Trace Back
    
    # Start from the highest value
    i, j = np.unravel_index(np.argmax(score), score.shape)
    
    # Traces
    trace1 = []
    trace2 = []
    
    # Find Paths
    m = 1
    while i > 0 or j > 0:
        
        # Diagonal
        if not (i > 0 and j > 0):
            diag = - np.inf
        else:
            diag = score[i - 1,j - 1] + match_score(seq1[j-1], seq2[i-1])
        
        # Left
        if not (j > 0):
            left = - np.inf
        else:
            left = score[i,j - 1] + gap_penalty
        
        # Up
        if not (i > 0):
            up = - np.inf
        else:
            up = score[i - 1,j] + gap_penalty
        
        # Max Score
        m = max(diag, left, up, 0)
        
        # Stop on 0
        if m == 0: break
        
        # Cases
        if diag == m:
            trace1.append(seq1[j-1])
            trace2.append(seq2[i-1])
            i -= 1
            j -= 1
        elif left == m:
            trace1.append(seq1[j-1])
            trace2.append("-")
            j -= 1
        elif up == m:
            trace1.append("-")
            trace2.append(seq2[i-1])
            i -= 1

    return score, (trace1[::-1], trace2[::-1])

# Example

In [13]:
match_award=2
gap_penalty=-2
mismatch_penalty=-1

seq1 = "GAATTCAGTTA"
seq2 = "GGATCGA"

matrix, traces = smith_waterman(seq1, seq2, match_award, gap_penalty, mismatch_penalty)

print(matrix)
print(traces[0])
print(traces[1])

[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 2 0 0 0]
 [0 2 1 0 0 0 0 0 2 1 0 0]
 [0 0 4 3 1 0 0 2 0 1 0 2]
 [0 0 2 3 5 3 1 0 1 2 3 1]
 [0 0 0 1 3 4 5 3 1 0 1 2]
 [0 2 0 0 1 2 3 4 5 3 1 0]
 [0 0 4 2 0 0 1 5 3 4 2 3]]
['G', 'A', 'A', 'T']
['G', 'G', 'A', 'T']
